# Applied Data Science Coursera  - Capstone Project

This notebook contains the code for the Capstone Project on Data Science by IBM and Coursera. To pick the best new office location in Stockholm, in this notebook I will use the Foursquare API and its __explore__ function to learn about Stockholm neighbourhoods and group the neighbourhoods into clusters using k-means clustering algorithm. Finally I will use the Folium library to visualize the neighbourhoods in Stockholm and their clusters.

### Importing needed libraries

In [1]:
import pandas as pd
import numpy as np
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#Importing plotting modules and map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.11.0 --yes 
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

### Downloading the list of neighbourhoods

In [40]:
places = pd.read_excel('Neighs.xlsx')
places

,Neighbourhood
0,"Abrahamsberg, Stockholm"
1,"Akalla, Stockholm"
2,"Akeshov, Stockholm"
3,"Alvik, Stockholm"
4,"Alvsjo, Stockholm"
...,...
62,"Ulvsunda, Stockholm"
63,"Vallingby, Stockholm"
64,"Vasaplan, Stockholm"
65,"Vasastan‎‎‎, Stockholm"


### Downloading the list of office space prices based on their distance from city centre

In [41]:
prices = pd.read_excel('Rent prices.xlsx')
prices

,Neighbourhood,Rent_psm,Distance from Centre
0,"Abrahamsberg, Stockholm",1275,2
1,"Akalla, Stockholm",1275,3
2,"Akeshov, Stockholm",1275,2
3,"Alvik, Stockholm",1275,2
4,"Alvsjo, Stockholm",1275,2
...,...,...,...
62,"Ulvsunda, Stockholm",1275,2
63,"Vallingby, Stockholm",1275,3
64,"Vasaplan, Stockholm",4950,1
65,"Vasastan‎‎‎, Stockholm",3300,1


### Creating empty data frame for Neighbourhoods, Latitude and Longitude

In [42]:
# defining the dataframe columns
column_names = ['Neighbourhood', 'Latitude', 'Longitude'] 

# creating the dataframe
neighbourhoods = pd.DataFrame(columns=column_names)

#showing dataframe
neighbourhoods

,Neighbourhood,Latitude,Longitude


### Getting the coordinates and populating them into the dataframe

In [43]:
neigh = places['Neighbourhood']
location = None
latitude = None
longitude = None

for data in range(0, len(neigh)):
    dt = neigh[data]
    
    geolocator = Nominatim(user_agent="sthlm_explorer")
    location = geolocator.geocode('{}'.format(dt))
    latitude = location.latitude
    longitude = location.longitude

    neighbourhoods = neighbourhoods.append({ 'Neighbourhood':dt,'Latitude': location.latitude,'Longitude': location.longitude}, ignore_index=True)

In [44]:
neighbourhoods

,Neighbourhood,Latitude,Longitude
0,"Abrahamsberg, Stockholm",59.336468,17.953763
1,"Akalla, Stockholm",59.414700,17.921818
2,"Akeshov, Stockholm",59.342139,17.924219
3,"Alvik, Stockholm",59.333401,17.982279
4,"Alvsjo, Stockholm",59.275849,18.001890
...,...,...,...
62,"Ulvsunda, Stockholm",59.337422,17.960066
63,"Vallingby, Stockholm",59.363934,17.870843
64,"Vasaplan, Stockholm",59.332433,18.057241
65,"Vasastan‎‎‎, Stockholm",59.341350,18.048947


### Creating a map to visualize all Stockholm's neighbourhoods

In [45]:
map_stockholm = folium.Map(location=[59.3293, 18.0686], zoom_start=11)

# add markers to map
for lat, lng, neighbourhood in zip(neighbourhoods['Latitude'], neighbourhoods['Longitude'], neighbourhoods['Neighbourhood']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        fill=True,
        fill_color='gold',
        fill_opacity=0.5,
        parse_html=True).add_to(map_stockholm)  
    
map_stockholm

### Defining Foursquare Credentials and Version

In [1]:
# @hidden_cell
CLIENT_ID = 'SHGJG3TRQG4JOOJ5LPSWG343WPZBCDTGVBVM53OKIB2P2DM0' # your Foursquare ID
CLIENT_SECRET = 'GWUMMLT4QRXWBK53XKSCP15SWZGLHBNOTDAN1WFJ3WYQLBI3' # your Foursquare Secret
VERSION = '20200619' # Foursquare API version

### Starting by exploring the first neighbourhood from the list

In [47]:
neighbourhood_latitude = neighbourhoods.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = neighbourhoods.loc[0, 'Longitude'] # neighbourhood longitude value

neighbourhood_name = neighbourhoods.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))


Latitude and longitude values of Abrahamsberg, Stockholm are 59.3364679, 17.9537633.


### Getting venues that are in Abrahamsberg within a radius of 700 meters.

In [60]:
LIMIT = 200 # limit of number of venues returned by Foursquare API
radius = 700 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)

In [61]:
import json # library to handle JSON files
import requests # library to handle requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe

### Getting the results

In [62]:
Abrahamsberg_results = requests.get(url).json()
Abrahamsberg_results

{'meta': {'code': 200, 'requestId': '5eecebbc618f43001bfe2e4c'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bromma',
  'headerFullLocation': 'Bromma, Stockholm',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 12,
  'suggestedBounds': {'ne': {'lat': 59.34276790630001,
    'lng': 17.96609329718361},
   'sw': {'lat': 59.33016789369999, 'lng': 17.94143330281639}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4be69a77910020a1d075d414',
       'name': "Daisy's",
       'location': {'address': 'Ulvsundavägen 71',
        'lat': 59.340587099504525,
        'lng': 17.96113959880651,
        'labeledLatLngs': [{'label': 'display',
          'lat': 59.340587099504525

### Defining function that extracts the category of the venue

In [63]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Transferring data from json into a pandas dataframe.

In [64]:
venues = Abrahamsberg_results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues

,name,categories,lat,lng
0,Daisy's,Fast Food Restaurant,59.340587,17.961140
1,Friskis & Svettis Abrahamsberg,Gym / Fitness Center,59.337785,17.948774
2,Pizzeria La Bella,Pizza Place,59.333929,17.947791
3,Brommagrillen,Fast Food Restaurant,59.342037,17.949059
4,ICA Abrahamsberg,Grocery Store,59.336172,17.952363
5,Sofie's Grill,Burger Joint,59.337958,17.952971
6,Abrahamsberg T-Bana,Metro Station,59.336617,17.953475
7,Skidsport,Sporting Goods Shop,59.337603,17.953325
8,Bonne Femme,Bakery,59.336121,17.952388
9,Texas Longhorn,Steakhouse,59.336699,17.956992


### Creating a function to repeat the same process for all the neighbourhoods in Stockholm

In [65]:
def getNearbyVenues(names, latitudes, longitudes, radius=700):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [66]:
stockholm_venues = getNearbyVenues(names=neighbourhoods['Neighbourhood'],
                                   latitudes=neighbourhoods['Latitude'],
                                   longitudes=neighbourhoods['Longitude']
                                  )

Abrahamsberg, Stockholm
Akalla, Stockholm
Akeshov, Stockholm
Alvik, Stockholm
Alvsjo, Stockholm
Appelviken, Stockholm 
Aspudden, Stockholm
Bagarmossen, Stockholm
Ballsta, Stockholm
Bandhagen, Stockholm
Beckomberga, Stockholm 
Bjorkhagen, Stockholm
Blackeberg, Stockholm
Bredang, Stockholm
Bromma, Stockholm
Bromsten, Stockholm
Djurgarden, Stockholm
Eneby, Stockholm
Enskede gard, Stockholm
Farsta, Stockholm
Fredhall‎, Stockholm
Fruangen, Stockholm‎  
Gamla Enskede, Stockholm
Gamla stan, Stockholm
Gardet, Stockholm‎ 
Grondal, Stockholm‎ 
Hagersten‎, Stockholm 
Hagerstensasen‎, Stockholm 
Hagsatra, Stockholm 
Herrangen, Stockholm
Hjorthagen, Stockholm 
Hornstull, Stockholm
Johanneshov, Stockholm
Kista, Stockholm
Kristineberg, Stockholm‎ 
Kungsholmen‎, Stockholm
Langholmen‎, Stockholm
Langsjo, Stockholm‎ 
Lidingo, Stockholm
Liljeholmen‎, Stockholm 
Lilla Essingen, Stockholm
Malarhojden, Stockholm
Marieberg, Stockholm‎‎ 
Mariehall, Stockholm
Midsommarkransen‎‎‎, Stockholm
Nacka, Stockholm
Nor

In [67]:
print(stockholm_venues.shape)
stockholm_venues.head()

(2096, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Abrahamsberg, Stockholm",59.336468,17.953763,Daisy's,59.340587,17.961140,Fast Food Restaurant
1,"Abrahamsberg, Stockholm",59.336468,17.953763,Friskis & Svettis Abrahamsberg,59.337785,17.948774,Gym / Fitness Center
2,"Abrahamsberg, Stockholm",59.336468,17.953763,Pizzeria La Bella,59.333929,17.947791,Pizza Place
3,"Abrahamsberg, Stockholm",59.336468,17.953763,Brommagrillen,59.342037,17.949059,Fast Food Restaurant
4,"Abrahamsberg, Stockholm",59.336468,17.953763,ICA Abrahamsberg,59.336172,17.952363,Grocery Store


### Checking how many venues were returned for each neighbourhood

In [68]:
stockholm_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Abrahamsberg, Stockholm",12,12,12,12,12,12
"Akalla, Stockholm",19,19,19,19,19,19
"Akeshov, Stockholm",8,8,8,8,8,8
"Alvik, Stockholm",22,22,22,22,22,22
"Alvsjo, Stockholm",27,27,27,27,27,27
...,...,...,...,...,...,...
"Ulvsunda, Stockholm",13,13,13,13,13,13
"Vallingby, Stockholm",23,23,23,23,23,23
"Vasaplan, Stockholm",100,100,100,100,100,100


### Checking how many unique categories can be found in all the returned venues

In [69]:
print('There are {} uniques categories.'.format(len(stockholm_venues['Venue Category'].unique())))

There are 249 uniques categories.


In [70]:
stockholm_venues['Venue Category'].unique()

array(['Fast Food Restaurant', 'Gym / Fitness Center', 'Pizza Place',
       'Grocery Store', 'Burger Joint', 'Metro Station',
       'Sporting Goods Shop', 'Bakery', 'Steakhouse', 'Lake',
       'Middle Eastern Restaurant', 'Pool', 'Supermarket', 'Plaza',
       'Asian Restaurant', 'Stadium', 'Pharmacy', 'Convenience Store',
       'Bus Stop', 'Candy Store', 'Bus Station', 'Farm', 'Moving Target',
       'Nature Preserve', 'Convention Center', 'Café',
       'Food & Drink Shop', 'Soccer Field', 'Scandinavian Restaurant',
       'Furniture / Home Store', 'Hawaiian Restaurant', 'Hotel',
       'Italian Restaurant', 'Gastropub', 'Park',
       'Modern European Restaurant', 'Thai Restaurant',
       'Light Rail Station', 'Martial Arts Dojo', 'Indian Restaurant',
       'Shopping Mall', 'Forest', 'Tennis Stadium', 'Sushi Restaurant',
       'Gym Pool', 'Restaurant', 'Sandwich Place', 'Coffee Shop',
       'Platform', 'Skating Rink', 'Track', 'Bistro', 'Bookstore',
       'Indie Movie Theat

### Analysing each neighbourhood

In [71]:
# one hot encoding
stockholm_onehot = pd.get_dummies(stockholm_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
stockholm_onehot['Neighbourhood'] = stockholm_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [stockholm_onehot.columns[-1]] + list(stockholm_onehot.columns[:-1])
stockholm_onehot = stockholm_onehot[fixed_columns]

stockholm_onehot.shape

(2096, 250)

In [72]:
stockholm_onehot.head(20)

,Neighbourhood,Accessories Store,Airport Gate,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Arcade,Art Gallery,Art Museum,...,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Weight Loss Center,Wine Bar,Women's Store,Yoga Studio,Zoo Exhibit
0,"Abrahamsberg, Stockholm",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Abrahamsberg, Stockholm",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Abrahamsberg, Stockholm",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Abrahamsberg, Stockholm",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Abrahamsberg, Stockholm",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,"Abrahamsberg, Stockholm",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,"Abrahamsberg, Stockholm",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"Abrahamsberg, Stockholm",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"Abrahamsberg, Stockholm",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,"Abrahamsberg, Stockholm",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
stockholm_onehot.tail(20)

,Neighbourhood,Accessories Store,Airport Gate,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Arcade,Art Gallery,Art Museum,...,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Weight Loss Center,Wine Bar,Women's Store,Yoga Studio,Zoo Exhibit
2076,"Vastberga‎, Stockholm",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2077,"Vastberga‎, Stockholm",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2078,"Vastberga‎, Stockholm",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2079,"Vastberga‎, Stockholm",0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2080,"Vastberga‎, Stockholm",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2081,"Vastberga‎, Stockholm",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2082,"Vastberga‎, Stockholm",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2083,"Vastberga‎, Stockholm",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2084,"Vastberga‎, Stockholm",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2085,"Vastberga‎, Stockholm",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Grouping rows by neighbourhood and by taking the mean of the frequency of occurrence of each venue category

In [74]:
stockholm_grouped = stockholm_onehot.groupby('Neighbourhood').mean().reset_index()
stockholm_grouped

,Neighbourhood,Accessories Store,Airport Gate,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Arcade,Art Gallery,Art Museum,...,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Weight Loss Center,Wine Bar,Women's Store,Yoga Studio,Zoo Exhibit
0,"Abrahamsberg, Stockholm",0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.00,...,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0
1,"Akalla, Stockholm",0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.00,...,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0
2,"Akeshov, Stockholm",0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.00,...,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0
3,"Alvik, Stockholm",0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.00,...,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0
4,"Alvsjo, Stockholm",0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.00,...,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,"Ulvsunda, Stockholm",0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.00,...,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0
63,"Vallingby, Stockholm",0.0,0.0,0.0,0.0,0.043478,0.0,0.00,0.000000,0.00,...,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0
64,"Vasaplan, Stockholm",0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.020000,0.00,...,0.00,0.0,0.00,0.01,0.01,0.0,0.01,0.0,0.01,0.0
65,"Vasastan‎‎‎, Stockholm",0.0,0.0,0.0,0.0,0.000000,0.0,0.01,0.000000,0.01,...,0.01,0.0,0.02,0.00,0.03,0.0,0.02,0.0,0.00,0.0


### Printing each neighbourhood along with the top 5 most common venues

In [75]:
num_top_venues = 5

for hood in stockholm_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = stockholm_grouped[stockholm_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Abrahamsberg, Stockholm----
                  venue  freq
0                Bakery  0.17
1  Fast Food Restaurant  0.17
2                  Lake  0.08
3          Burger Joint  0.08
4   Sporting Goods Shop  0.08


----Akalla, Stockholm----
                  venue  freq
0                  Farm  0.11
1                 Plaza  0.11
2  Fast Food Restaurant  0.11
3           Pizza Place  0.05
4         Moving Target  0.05


----Akeshov, Stockholm----
               venue  freq
0    Nature Preserve  0.25
1       Soccer Field  0.12
2  Convention Center  0.12
3               Pool  0.12
4  Food & Drink Shop  0.12


----Alvik, Stockholm----
                venue  freq
0     Thai Restaurant  0.09
1         Pizza Place  0.09
2               Hotel  0.09
3  Light Rail Station  0.09
4                Park  0.05


----Alvsjo, Stockholm----
              venue  freq
0              Café  0.07
1             Hotel  0.07
2       Supermarket  0.07
3    Tennis Stadium  0.04
4  Sushi Restaurant  0.04


----Appe

### Transferring that data into a dataframe

1. Function to sort the venues in descending order.

In [76]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

2. Creating a new dataframe to display the top 10 venues for each neighborhood.

In [77]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = stockholm_grouped['Neighbourhood']

for ind in np.arange(stockholm_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(stockholm_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted  

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Abrahamsberg, Stockholm",Bakery,Fast Food Restaurant,Metro Station,Grocery Store,Gym / Fitness Center,Lake,Steakhouse,Burger Joint,Sporting Goods Shop,Pizza Place
1,"Akalla, Stockholm",Plaza,Fast Food Restaurant,Farm,Convenience Store,Grocery Store,Candy Store,Moving Target,Supermarket,Pool,Stadium
2,"Akeshov, Stockholm",Nature Preserve,Convention Center,Food & Drink Shop,Metro Station,Soccer Field,Pool,Café,Falafel Restaurant,Food Truck,Food Court
3,"Alvik, Stockholm",Hotel,Thai Restaurant,Pizza Place,Light Rail Station,Convenience Store,Grocery Store,Bakery,Martial Arts Dojo,Furniture / Home Store,Scandinavian Restaurant
4,"Alvsjo, Stockholm",Café,Supermarket,Hotel,Forest,Pizza Place,Bakery,Plaza,Sushi Restaurant,Restaurant,Platform
...,...,...,...,...,...,...,...,...,...,...,...
62,"Ulvsunda, Stockholm",Metro Station,Burger Joint,Bakery,Fast Food Restaurant,Grocery Store,Gym / Fitness Center,Steakhouse,Castle,Lake,Sporting Goods Shop
63,"Vallingby, Stockholm",Café,Gym / Fitness Center,Coffee Shop,Grocery Store,Liquor Store,Restaurant,Bookstore,Bus Station,Movie Theater,Pool
64,"Vasaplan, Stockholm",Hotel,Café,Scandinavian Restaurant,Burger Joint,Gym / Fitness Center,Bakery,Asian Restaurant,Bookstore,Salad Place,Ice Cream Shop
65,"Vasastan‎‎‎, Stockholm",Café,Scandinavian Restaurant,Sushi Restaurant,Bakery,Pizza Place,French Restaurant,Coffee Shop,Italian Restaurant,Burger Joint,Park


### Adding the office prices and distance from city to the Stockholm dataframe

In [78]:
stockholm_grouped = pd.merge(prices, stockholm_grouped, on = 'Neighbourhood')
stockholm_grouped

,Neighbourhood,Rent_psm,Distance from Centre,Accessories Store,Airport Gate,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Arcade,...,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Weight Loss Center,Wine Bar,Women's Store,Yoga Studio,Zoo Exhibit
0,"Abrahamsberg, Stockholm",1275,2,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,...,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0
1,"Akalla, Stockholm",1275,3,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,...,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0
2,"Akeshov, Stockholm",1275,2,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,...,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0
3,"Alvik, Stockholm",1275,2,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,...,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0
4,"Alvsjo, Stockholm",1275,2,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,...,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,"Ulvsunda, Stockholm",1275,2,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,...,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0
63,"Vallingby, Stockholm",1275,3,0.0,0.0,0.0,0.0,0.043478,0.0,0.00,...,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0
64,"Vasaplan, Stockholm",4950,1,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,...,0.00,0.0,0.00,0.01,0.01,0.0,0.01,0.0,0.01,0.0
65,"Vasastan‎‎‎, Stockholm",3300,1,0.0,0.0,0.0,0.0,0.000000,0.0,0.01,...,0.01,0.0,0.02,0.00,0.03,0.0,0.02,0.0,0.00,0.0


## Clustering the neighbourhoods

In [79]:
# import silhuette score
from sklearn.metrics import silhouette_samples, silhouette_score

### Selecting optimal number of clusters using the silhouette score:

In [80]:
stockholm_grouped_clustering = stockholm_grouped.drop('Neighbourhood', 1)
stockholm_grouped_clustering.head()

,Rent_psm,Distance from Centre,Accessories Store,Airport Gate,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Arcade,Art Gallery,...,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Weight Loss Center,Wine Bar,Women's Store,Yoga Studio,Zoo Exhibit
0,1275,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1275,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1275,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1275,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1275,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [81]:
# Choosing optimal number of clusters with silhuette score

range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10]
for n_clusters in range_n_clusters:
    clusterer = KMeans(n_clusters=n_clusters)
    preds = clusterer.fit_predict(stockholm_grouped_clustering)
    centers = clusterer.cluster_centers_

    score = silhouette_score(stockholm_grouped_clustering, preds)
    print("For n_clusters = {}, silhouette score is {})".format(n_clusters, score))

For n_clusters = 2, silhouette score is 0.8796550353738828)
For n_clusters = 3, silhouette score is 0.8991307384475417)
For n_clusters = 4, silhouette score is 0.9111182940897871)
For n_clusters = 5, silhouette score is 0.914295610351762)
For n_clusters = 6, silhouette score is 0.9324578034514397)
For n_clusters = 7, silhouette score is 0.9289001490808191)
For n_clusters = 8, silhouette score is 0.9249866869440991)
For n_clusters = 9, silhouette score is 0.9094836731710633)
For n_clusters = 10, silhouette score is 0.9067090725921547)


The optimal number of clusters k is the one that maximize the average silhouette over a range of possible values for k. The above scores suggest an optimal of 6 clusters.

### Running *k*-means to cluster the neighbourhood into 6 clusters:

In [82]:
# set number of clusters
kclusters = 6

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(stockholm_grouped_clustering)

### Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighbourhood:

In [83]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

stockholm_merged = neighbourhoods

#merge dataframes to add latitude/longitude for each neighbourhood
stockholm_merged = stockholm_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

stockholm_merged 

,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Abrahamsberg, Stockholm",59.336468,17.953763,0,Bakery,Fast Food Restaurant,Metro Station,Grocery Store,Gym / Fitness Center,Lake,Steakhouse,Burger Joint,Sporting Goods Shop,Pizza Place
1,"Akalla, Stockholm",59.414700,17.921818,0,Plaza,Fast Food Restaurant,Farm,Convenience Store,Grocery Store,Candy Store,Moving Target,Supermarket,Pool,Stadium
2,"Akeshov, Stockholm",59.342139,17.924219,0,Nature Preserve,Convention Center,Food & Drink Shop,Metro Station,Soccer Field,Pool,Café,Falafel Restaurant,Food Truck,Food Court
3,"Alvik, Stockholm",59.333401,17.982279,0,Hotel,Thai Restaurant,Pizza Place,Light Rail Station,Convenience Store,Grocery Store,Bakery,Martial Arts Dojo,Furniture / Home Store,Scandinavian Restaurant
4,"Alvsjo, Stockholm",59.275849,18.001890,0,Café,Supermarket,Hotel,Forest,Pizza Place,Bakery,Plaza,Sushi Restaurant,Restaurant,Platform
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,"Ulvsunda, Stockholm",59.337422,17.960066,0,Metro Station,Burger Joint,Bakery,Fast Food Restaurant,Grocery Store,Gym / Fitness Center,Steakhouse,Castle,Lake,Sporting Goods Shop
63,"Vallingby, Stockholm",59.363934,17.870843,0,Café,Gym / Fitness Center,Coffee Shop,Grocery Store,Liquor Store,Restaurant,Bookstore,Bus Station,Movie Theater,Pool
64,"Vasaplan, Stockholm",59.332433,18.057241,4,Hotel,Café,Scandinavian Restaurant,Burger Joint,Gym / Fitness Center,Bakery,Asian Restaurant,Bookstore,Salad Place,Ice Cream Shop
65,"Vasastan‎‎‎, Stockholm",59.341350,18.048947,1,Café,Scandinavian Restaurant,Sushi Restaurant,Bakery,Pizza Place,French Restaurant,Coffee Shop,Italian Restaurant,Burger Joint,Park


## Visualizing the resulting clusters:

In [84]:
# create map
map_clusters_sthlm = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(stockholm_merged['Latitude'], stockholm_merged['Longitude'], stockholm_merged['Neighbourhood'], stockholm_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_sthlm)

map_clusters_sthlm       

## Examine Clusters

In [89]:
stockholm_merged = pd.merge(prices, stockholm_merged, on = 'Neighbourhood')

### Cluster one

In [90]:
stockholm_merged.loc[stockholm_merged['Cluster Labels'] == 0, stockholm_merged.columns[[0] + list(range(5, stockholm_merged.shape[1]))]]

,Neighbourhood,Rent_psm_y,Distance from Centre_y,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Abrahamsberg, Stockholm",1275,2,59.336468,17.953763,0,Bakery,Fast Food Restaurant,Metro Station,Grocery Store,Gym / Fitness Center,Lake,Steakhouse,Burger Joint,Sporting Goods Shop,Pizza Place
1,"Akalla, Stockholm",1275,3,59.414700,17.921818,0,Plaza,Fast Food Restaurant,Farm,Convenience Store,Grocery Store,Candy Store,Moving Target,Supermarket,Pool,Stadium
2,"Akeshov, Stockholm",1275,2,59.342139,17.924219,0,Nature Preserve,Convention Center,Food & Drink Shop,Metro Station,Soccer Field,Pool,Café,Falafel Restaurant,Food Truck,Food Court
3,"Alvik, Stockholm",1275,2,59.333401,17.982279,0,Hotel,Thai Restaurant,Pizza Place,Light Rail Station,Convenience Store,Grocery Store,Bakery,Martial Arts Dojo,Furniture / Home Store,Scandinavian Restaurant
4,"Alvsjo, Stockholm",1275,2,59.275849,18.001890,0,Café,Supermarket,Hotel,Forest,Pizza Place,Bakery,Plaza,Sushi Restaurant,Restaurant,Platform
5,"Appelviken, Stockholm",1275,2,59.326225,17.971132,0,Thai Restaurant,Track,Bakery,Light Rail Station,Grocery Store,Restaurant,Skating Rink,Hotel,Food & Drink Shop,Food
6,"Aspudden, Stockholm",1275,2,59.306466,18.001373,0,Pizza Place,Park,Café,Sushi Restaurant,Thai Restaurant,Grocery Store,Metro Station,Sandwich Place,Bookstore,Bistro
7,"Bagarmossen, Stockholm",1275,2,59.276186,18.131340,0,Café,Grocery Store,Bakery,Park,Scandinavian Restaurant,Pizza Place,Sushi Restaurant,Plaza,Pub,Supermarket
8,"Ballsta, Stockholm",1275,3,59.363309,17.925723,0,Scandinavian Restaurant,Auto Workshop,Playground,Gym,Flower Shop,Zoo Exhibit,Fast Food Restaurant,Falafel Restaurant,Farm,Farmers Market
9,"Bandhagen, Stockholm",1275,2,59.270305,18.049588,0,Bus Station,Café,Pizza Place,Hobby Shop,Playground,Stadium,Grocery Store,Farm,Farmers Market,Fast Food Restaurant


### Cluster two

In [91]:
stockholm_merged.loc[stockholm_merged['Cluster Labels'] == 1, stockholm_merged.columns[[0] + list(range(5, stockholm_merged.shape[1]))]]

,Neighbourhood,Rent_psm_y,Distance from Centre_y,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,"Fredhall‎, Stockholm",3300,1,59.329822,18.003028,1,Café,Beach,Pizza Place,Thai Restaurant,Park,Track,Boat or Ferry,Restaurant,Grocery Store,Mini Golf
23,"Gamla stan, Stockholm",3300,1,59.324953,18.070643,1,Scandinavian Restaurant,Plaza,History Museum,Hotel,Pub,Boat or Ferry,Café,Restaurant,Cocktail Bar,Bakery
24,"Gardet, Stockholm‎",3300,1,59.344243,18.090890,1,Bakery,Thai Restaurant,Stadium,Café,Sushi Restaurant,Greek Restaurant,Restaurant,Park,Coffee Shop,Chocolate Shop
25,"Grondal, Stockholm‎",3300,1,59.315542,18.010912,1,Bakery,Beach,Sushi Restaurant,Food Truck,Persian Restaurant,Bus Stop,Indian Restaurant,Pizza Place,Café,Other Nightlife
31,"Hornstull, Stockholm",3300,1,59.315826,18.034817,1,Italian Restaurant,Bar,Café,Harbor / Marina,Playground,Chinese Restaurant,Gym / Fitness Center,Park,Middle Eastern Restaurant,Sushi Restaurant
35,"Kungsholmen‎, Stockholm",3300,1,59.332742,18.023772,1,Café,Bakery,Japanese Restaurant,Sushi Restaurant,Pizza Place,Park,Supermarket,Gym / Fitness Center,Scandinavian Restaurant,Hotel
39,"Liljeholmen‎, Stockholm",3300,1,59.310612,18.022832,1,Café,Gym / Fitness Center,Thai Restaurant,Bakery,Bar,Asian Restaurant,Pizza Place,Restaurant,Chinese Restaurant,Supermarket
50,"Riddarholmen‎, Stockholm",3300,1,59.325016,18.062353,1,Scandinavian Restaurant,Café,Cocktail Bar,History Museum,Plaza,Pub,Bakery,Wine Bar,Park,Hotel
53,"Skeppsholmen, Stockholm",3300,1,59.325356,18.085004,1,Scandinavian Restaurant,Theme Park Ride / Attraction,Café,Museum,Art Museum,Coffee Shop,Hotel,History Museum,Restaurant,Hostel
55,"Sodermalm‎, Stockholm",3200,1,59.312378,18.069756,1,Coffee Shop,Bakery,Café,Indian Restaurant,Arts & Crafts Store,Burger Joint,Bookstore,Record Shop,Clothing Store,Asian Restaurant


### Cluster three

In [92]:
stockholm_merged.loc[stockholm_merged['Cluster Labels'] == 2, stockholm_merged.columns[[0] + list(range(5, stockholm_merged.shape[1]))]]

,Neighbourhood,Rent_psm_y,Distance from Centre_y,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
47,"Norrmalm‎, Stockholm",3800,1,59.334082,18.062905,2,Hotel,Scandinavian Restaurant,Café,Burger Joint,Gym / Fitness Center,Italian Restaurant,Clothing Store,Movie Theater,Department Store,Bakery
48,"Ostermalm, Stockholm",4100,1,59.336723,18.085918,2,Scandinavian Restaurant,Italian Restaurant,Café,Hotel,Sushi Restaurant,French Restaurant,Bakery,Pizza Place,Bar,Thai Restaurant


### Cluster four

In [93]:
stockholm_merged.loc[stockholm_merged['Cluster Labels'] == 3, stockholm_merged.columns[[0] + list(range(5, stockholm_merged.shape[1]))]]

,Neighbourhood,Rent_psm_y,Distance from Centre_y,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"Bromma, Stockholm",1950,2,59.340065,17.933115,3,Supermarket,Soccer Field,Gym / Fitness Center,Café,Sushi Restaurant,Pub,Pool,Tennis Court,Metro Station,Convention Center
33,"Kista, Stockholm",1950,3,59.403311,17.944285,3,Asian Restaurant,Hotel,Gym / Fitness Center,Restaurant,Café,Middle Eastern Restaurant,Coffee Shop,Scandinavian Restaurant,Grocery Store,Sandwich Place
38,"Lidingo, Stockholm",2200,2,59.365112,18.195643,3,Mountain,Bathing Area,Golf Course,Zoo Exhibit,Fried Chicken Joint,Fraternity House,Forest,Food Truck,Food Court,Food & Drink Shop
56,"Sodra Hammarbyhamnen, Stockholm",2200,2,59.304181,18.098584,3,Sushi Restaurant,Café,Scandinavian Restaurant,Hotel,Restaurant,Bakery,Playground,Gym / Fitness Center,Asian Restaurant,Convenience Store
57,"Solna, Stockholm",2100,2,59.361367,18.001369,3,Supermarket,Pharmacy,Indian Restaurant,Bakery,Burger Joint,Pizza Place,Café,Department Store,Convenience Store,Clothing Store


### Cluster five

In [94]:
stockholm_merged.loc[stockholm_merged['Cluster Labels'] == 4, stockholm_merged.columns[[0] + list(range(5, stockholm_merged.shape[1]))]]

,Neighbourhood,Rent_psm_y,Distance from Centre_y,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,"Vasaplan, Stockholm",4950,1,59.332433,18.057241,4,Hotel,Café,Scandinavian Restaurant,Burger Joint,Gym / Fitness Center,Bakery,Asian Restaurant,Bookstore,Salad Place,Ice Cream Shop


### Cluster six

In [95]:
stockholm_merged.loc[stockholm_merged['Cluster Labels'] == 5, stockholm_merged.columns[[0] + list(range(5, stockholm_merged.shape[1]))]]

,Neighbourhood,Rent_psm_y,Distance from Centre_y,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,"Langholmen‎, Stockholm",3000,1,59.321837,18.027465,5,Italian Restaurant,Beach,Café,Harbor / Marina,Hotel,Bar,Pizza Place,Lake,Event Space,Spanish Restaurant
40,"Lilla Essingen, Stockholm",3000,1,59.324538,18.006317,5,Grocery Store,Pizza Place,Scandinavian Restaurant,Restaurant,Café,Beach,Pier,Thai Restaurant,Harbor / Marina,Deli / Bodega
59,"Stora Essingen, Stockholm",3000,1,59.321747,17.990692,5,Café,Bus Stop,Supermarket,Asian Restaurant,Italian Restaurant,Plaza,French Restaurant,Bus Station,IT Services,Deli / Bodega
